In [ ]:
import os
import pickle
import cPickle as cpkl
import csv
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

# mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['savefig.dpi'] = 250
# mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'

import numpy as np
import scipy.stats as sps

import chippr
import chippr.plot_utils as pu

%matplotlib inline

# Examining CosmoLike input

In [ ]:
# print(np.shape(cl_input))
# n_tomobins = np.shape(cl_input)[1] - 1
# 1 column of redshifts, 4 tomographic bins, 350 redshift bins
# too many redshift bins, I'll set the truth to that binned down

In [ ]:
cl_input = np.genfromtxt('../results/single/results/nz_histo.txt')
n_tomobins = np.shape(cl_input)[1] - 1
fine_dif = np.mean(cl_input.T[0])
for i in range(n_tomobins):
    plt.plot(cl_input.T[0], cl_input.T[i+1])
    print(np.sum(cl_input.T[i+1] * fine_dif))

In [ ]:
# this is the truth!
cl_data = np.empty((35, 5))
for i in range(35):
    cl_data[i][1:] = np.sum(cl_input[i * 10:(i + 1) * 10, 1:], axis=0)
    cl_data[i][0] = cl_input[i * 10][0]
for i in range(n_tomobins):
    plt.plot(cl_data.T[0], cl_data.T[i+1])
    
np.savetxt('../results/CosmoLike/nz_format_test.txt', cl_data)

# make CHIPPR output into CosmoLike format

## truth

in units of counts

In [ ]:
bin_ends = np.genfromtxt('../results/0single_lsst/data/metadata.txt')
bin_difs = bin_ends[1:] - bin_ends[:-1]
bin_mids = (bin_ends[1:] + bin_ends[:-1]) / 2.

In [ ]:
all_true_nzs = [bin_mids]
for i in range(n_tomobins):
    true_zs = np.genfromtxt('../results/' + str(i) + 'single_lsst/data/true_vals.txt').T[0]
    true_nz = np.histogram(true_zs, bins=bin_ends)[0] / float(len(true_zs)) / bin_difs
    all_true_nzs.append(true_nz)
all_true_nzs = np.array(all_true_nzs)
np.savetxt('../results/CosmoLike/true_nz.txt', all_true_nzs.T)

In [ ]:
for i in range(n_tomobins):
    plt.plot(all_true_nzs[0], all_true_nzs[i+1])

## estimators

log n(z) piecewise constant and separate files per tomobin each with all formats

need single file with z values, point evaluations in each tomobin

In [ ]:
all_stk_nzs, all_mle_nzs, all_map_nzs = [bin_mids], [bin_mids], [bin_mids]
for i in range(n_tomobins):
    with open('../results/' + str(i) + 'single_lsst/results/nz.p', 'rb') as test_file:
        test_info = cpkl.load(test_file)
    all_stk_nzs.append(np.exp(test_info['estimators']['log_stacked_nz']))
    all_mle_nzs.append(np.exp(test_info['estimators']['log_mmle_nz']))
    all_map_nzs.append(np.exp(test_info['estimators']['log_mmap_nz']))
all_stk_nzs = np.array(all_stk_nzs)
all_mle_nzs = np.array(all_mle_nzs)
all_map_nzs = np.array(all_map_nzs)
np.savetxt('../results/CosmoLike/stack_nz.txt', all_stk_nzs.T)
np.savetxt('../results/CosmoLike/mmle_nz.txt', all_mle_nzs.T)
np.savetxt('../results/CosmoLike/mmap_nz.txt', all_map_nzs.T)

In [ ]:
for i in range(n_tomobins):
    plt.plot(all_true_nzs[0], all_true_nzs[i+1], c='k')
    plt.plot(all_true_nzs[0], all_stk_nzs[i+1], c='g')
    plt.plot(all_true_nzs[0], all_mle_nzs[i+1], c='b')
    plt.plot(all_true_nzs[0], all_map_nzs[i+1], c='r')
    

# what to do with CosmoLike output

In [ ]:
testnames = ['true_nz', 'stack_nz', 'mmle_nz', 'mmap_nz']

In [ ]:
all_covariances = []
all_invcovariances = []
magfactors = []
for testname in testnames:
    covariances = np.zeros((200, 200)) + sys.float_info.min
#     with open(os.path.join('../results/CosmoLike', 'Cl_cov.nz'+testname+'.txt'), 'rb') as cosmolike_file:
#         cosmolike_reader = csv.reader(cosmolike_file, delimiter=' ')
#         cosmolike_reader.next()
#         for row in cosmolike_reader:
#             # covariance matrix is filled with positive values << floating point precision.
#             # I'm going to inflate them and deflate before reporting results
#             covariances[int(row[0])][int(row[1])] = float(row[2])
#             covariances[int(row[1])][int(row[0])] = float(row[2])
    covariance_table = np.genfromtxt('../results/CosmoLike/Cl_cov.'+testname+'.txt')
    magfactor = np.asarray(np.min(covariance_table.T[-1]))
    print(magfactor)
    magfactors.append(magfactor)
    for row in covariance_table:
#         print(row)
        covariances[int(row[0])][int(row[1])] = row[2]# / magfactor
        covariances[int(row[1])][int(row[0])] = row[2]# / magfactor
#     covariances = covariances - sys.float_info.min
#     assert(np.all(covariances >= 0.))
#     covariances += sys.float_info.min
#     covariances = covariances / magfactor
    assert(np.all(covariances > 0.))
    all_covariances.append(covariances)
    invcovariances = np.linalg.inv(covariances)
    mask = np.where((invcovariances < sys.float_info.min))
    invcovariances[mask] = sys.float_info.min
    print(np.shape(invcovariances))
    assert(np.all(invcovariances > 0.))
    all_invcovariances.append(invcovariances)

In [ ]:
# # plt.imshow(np.log(covariances))
# # print(covariances)

# invcovariances = np.linalg.pinv(covariances)# / sys.float_info.epsilon) * sys.float_info.epsilon
# # print(invcovariances)

# # # invcovariances = np.linalg.pinv(1.e15 * covariances) * 1.e15
# # # print(invcovariances)

# # plt.imshow(np.log(invcovariances * sys.float_info.epsilon))
# # print(invcovariances * sys.float_info.epsilon)

In [ ]:
# Cl, dCldOm, dClds8, dCldns, dCldw0, dCldwa, dCldOb, dCldH0 = np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200)
all_deriv_info = []
for testname in testnames:
#     deriv_info = np.zeros((200, 7)) + sys.float_info.min
#     with open(os.path.join('../results/CosmoLike', 'Cl_derivs.nz'+testname+'.txt'), 'rb') as cosmolike_file:
#         cosmolike_reader = csv.reader(cosmolike_file, delimiter=' ')
#         cosmolike_reader.next()
#         i = 0
#         while i < 200:
#             for row in cosmolike_reader:
#                 for j in range(7):
#                     deriv_info[i][j] = float(row[j])
#             i += 1
    deriv_info = np.genfromtxt('../results/CosmoLike/Cl_derivs.'+testname+'.txt')#, skip_header=0)
#     print(np.shape(deriv_info))
    deriv_info = np.vstack((np.zeros(7), deriv_info))
    deriv_info = deriv_info.T
    all_deriv_info.append(deriv_info)

In [ ]:
# # deriv_info = deriv_info.T
# dCldOm, dClds8, dCldns, dCldw0, dCldwa, dCldOb, dCldH0 = deriv_info[0], deriv_info[1], deriv_info[2], deriv_info[3], deriv_info[4], deriv_info[5], deriv_info[6]

In [ ]:
all_fisher = []
all_invfisher = []
for k in range(len(testnames)):
    ls = np.arange(200)
    fisher = np.eye(7)
    for i in range(7):
        for j in range(i+1):
            fisher[i][j] = np.sum((2. * ls[1:] + 1.) * all_deriv_info[k][i][1:] * 
                                  (all_invcovariances[k][1:, 1:]) * all_deriv_info[k][j][1:])
            fisher[j][i] = fisher[i][j]
    all_fisher.append(fisher)
    invfisher = np.linalg.pinv(fisher)#np.linalg.inv(fisher/sys.float_info.epsilon) * sys.float_info.epsilon
    all_invfisher.append(invfisher)

In [ ]:
# # print(fisher/sys.float_info.epsilon)
# # plt.imshow(fisher/sys.float_info.epsilon)

# invfisher = np.linalg.inv(fisher/sys.float_info.epsilon) * sys.float_info.epsilon
# plt.imshow(np.log(all_fisher[0]))
# plt.colorbar()
# print(invfisher)

In [ ]:
colors = ['k', pu.colors[1], pu.colors[0], pu.colors[2]] 
styles = [[(0, (1, 0.001))], [(0, (2, 2))], [(0, (1, 2))], [(0, (2, 1))]]
# names = ['true', 'stack', 'CHIPPR', 'MAP']
keys = ['dCl/dOm', 'dCl/ds8', 'dCl/dns', 'dCl/dw0', 'dCl/dwa', 'dCl/dOb', 'dCl/dH0']
# keys = [r'$\Omega_{m}$', r'$S_{8}$', r'$n_{s}$', r'$w_{0}$', r'$w_{a}$', r'$\Omega_{b}$', r'$H_{0}$']

In [ ]:
def make_ellipse_params(fisher):
    diag_elems = np.diag(fisher)
    term1 = (diag_elems[:, np.newaxis] + diag_elems[np.newaxis, :]) / 2.
#     print(term1)
    term2 = np.sqrt((diag_elems[:, np.newaxis] - diag_elems[np.newaxis, :])**2 / 4. + fisher * fisher.T)
#     print(term2)
    a = np.sqrt(term1 + term2)# (added abs?)
    b = np.sqrt(np.abs(term1 - term2))# (added abs?)
#     print(b)
    t = np.arctan((fisher + fisher.T) / (diag_elems[:, np.newaxis] - diag_elems[np.newaxis, :])) / 2.
    assert(np.all(b >= 0.))
    return(a, b, t)

fish_params = []
for k in range(len(testnames)):
    fisher = all_invfisher[k]
    (a, b, t) = make_ellipse_params(fisher)
    fish_params.append((a, b, t))
fish_params = np.array(fish_params)

In [ ]:
def mycorner(fish_params, keys):
    
    ncol = len(keys)
    fig = plt.figure(figsize=(ncol*5, ncol*5))
    ax = [[fig.add_subplot(ncol, ncol, ncol * i + j + 1) for j in range(i+1)] for i in range(ncol)]
    to_keep = range(ncol)#[0, 1, 2, 4]
    
    extrema = np.zeros(ncol)
    for k in range(len(testnames)):
        axl = fig.add_subplot(ncol, ncol, ncol)
        axl.text(0.5, 0.75 - 0.1 * k, testnames[k], fontsize=16, color=colors[k])
#         fisher = fishers_info[k]
        
        a, b, t = fish_params[k]
        extrema = np.max(np.sqrt(a), axis=0) / 2.
        
        for i in range(ncol):#range(len(to_keep)):#range(ncol):
#             sxx = fisher[i][i]
#             sx = np.sqrt(sxx)# (added abs!)
#             extrema[i] = np.max((sx, extrema[i]))
    #         i = k#to_keep[k]
            for j in range(i + 1):#to_keep[:k]:#range(i+1):
#                 ax[i][j].set_xlim(-1. * extrema[i], extrema[i])
                if i == j:
                    lim_val = np.sqrt(all_invfisher[k][i][i])
                    x_grid = np.linspace(-3. * lim_val, 3. * lim_val, 100)
                    func = sps.norm(0., lim_val)# (added abs!)
                    ax[i][j].plot(x_grid, func.pdf(x_grid), 
                                  color=colors[k], label=testnames[k], alpha=0.5, linestyle=styles[k][0], linewidth=3.)
                    ax[i][j].set_xlabel(keys[i])
                    ax[i][j].set_yticks([])
                else:
#                     syy = fisher[j][j]
#                     sy = np.sqrt(syy)# (added abs!)
#                     ylim = (-5.*sy, 5.*sy)
#                     term1 = (sxx + syy)/2.
#                     term2 = np.sqrt((sxx - syy)**2/4. + fisher[i][j] * fisher[j][i])# (added abs!)
#                     a = np.sqrt(np.abs(term1 + term2))# (added abs?)
#                     b = np.sqrt(np.abs(term1 - term2))# (added abs?)
#                     t = np.arctan((fisher[i][j] + fisher[j][i])/(sxx-syy))/2.
#                     print((k, i, j, term1-term2, a, b))
                    ellipse = Ellipse(xy=(0., 0.), width=2.*a[i][j], height=2.*b[i][j], angle=t[i][j]*180./np.pi, 
                                      alpha=0.25, color=colors[k], linestyle=styles[k][0], lw=2.)
                    ax[i][j].add_artist(ellipse)
                    ax[i][j].set_xlabel(keys[i])
                    ax[i][j].set_ylabel(keys[j])
                    ax[i][j].set_ylim(-1.* extrema[i], 1. * extrema[i])
                    ax[i][j].set_xlim(-1. * extrema[i], extrema[i])
#                     ax[i][j].set_xlim(-4.*a, 4.*a)
#                     ax[i][j].set_ylim(-4.*b, 4.*b)
#                 ax[i][j].set_xlim(-1. * extrema[j], extrema[j])
    
    plt.savefig('../results/CosmoLike/final_plot.png', dpi=250)
    return

# mycorner(all_invfisher, keys)

In [ ]:
mycorner(fish_params, keys)

# making CosmoLike input

In [ ]:
test_dir = '../results/single'

In [ ]:
simulated_posteriors = chippr.catalog(params='single.txt', loc=test_dir)

In [ ]:
data = simulated_posteriors.read(loc='data', style='.txt')

In [ ]:
params = chippr.utils.ingest('single.txt')
def check_prob_params(params):
    """
    Sets parameter values pertaining to components of probability

    Parameters
    ----------
    params: dict
        dictionary containing key/value pairs for probability

    Returns
    -------
    params: dict
        dictionary containing key/value pairs for probability
    """
    if 'prior_mean' not in params:
        params['prior_mean'] = 'interim'
    else:
        params['prior_mean'] = params['prior_mean'][0]
    if 'no_prior' not in params:
        params['no_prior'] = 0
    else:
        params['no_prior'] = int(params['no_prior'][0])
    if 'no_data' not in params:
        params['no_data'] = 0
    else:
        params['no_data'] = int(params['no_data'][0])
    return params
params = check_prob_params(params)
def set_up_prior(data, params):
    """
    Function to create prior distribution from data

    Parameters
    ----------
    data: dict
        catalog dictionary containing bin endpoints, log interim prior, and log
        interim posteriors
    params: dict
        dictionary of parameter values for creation of prior

    Returns
    -------
    prior: chippr.mvn object
        prior distribution as multivariate normal
    """
    zs = data['bin_ends']
    log_nz_intp = data['log_interim_prior']
    log_z_posts = data['log_interim_posteriors']

    z_difs = zs[1:]-zs[:-1]
    z_mids = (zs[1:]+zs[:-1])/2.
    n_bins = len(z_mids)

    n_pdfs = len(log_z_posts)

    a = 1.# / n_bins
    b = 20.#1. / z_difs ** 2
    c = a / n_pdfs
    prior_var = np.eye(n_bins)
    for k in range(n_bins):
        prior_var[k] = a * np.exp(-0.5 * b * (z_mids[k] - z_mids) ** 2)
    prior_var += c * np.identity(n_bins)

    prior_mean = log_nz_intp
    prior = chippr.mvn(prior_mean, prior_var)
    if params['prior_mean'] == 'sample':
        new_mean = prior.sample_one()
        prior = chippr.mvn(new_mean, prior_var)
        print(params['prior_mean'], prior_mean, new_mean)
    else:
        print(params['prior_mean'], prior_mean)

    return (prior, prior_var)
(prior, cov) = set_up_prior(data, params)

In [ ]:
with open(os.path.join('../results/single/data', 'true_params.p'), 'r') as true_file:
    true_nz_info = pickle.load(true_file)

In [ ]:
true_nz_info['amps']

In [ ]:
true_funcs = [chippr.discrete(np.array([true_nz_info['bins'][0], true_nz_info['bins'][-1]]), true_nz_info['amps'])]
true_amps = true_nz_info['amps']
# # print(true_amps)
true_nz = true_funcs[0]#chippr.gmix(true_amps, true_funcs, limits=(min(true_nz_info['bins']), max(true_nz_info['bins'])))
grid_mids = (true_nz_info['bins'][1:] + true_nz_info['bins'][:-1])/2.
plt.plot(grid_mids, true_nz.evaluate(grid_mids))


In [ ]:
nz = chippr.log_z_dens(data, prior, truth=true_nz, loc='../results/single', vb=True)

In [ ]:
nz.read('nz.p')

In [ ]:
bins_to_write = np.linspace(0.0101, 3.5001, 350)
empty_bins = np.random.random((350, 4))/350.


In [ ]:
bin_mids = (nz.info['bin_ends'][1:] + nz.info['bin_ends'][:-1])/2.
with open(os.path.join('../results/single/results', 'nz_mmle_test.txt'), 'wb') as cosmolike_file:
#     cosmolike_file.write(zip(bin_mids, np.exp(nz.info['estimators']['log_mmle_nz']))
    cosmolike_writer = csv.writer(cosmolike_file, delimiter=' ')
    cosmolike_writer.writerows(zip(bin_mids, np.exp(nz.info['estimators']['log_mmle_nz'])))

# Placeholder

# Revisiting!

How to run cosmolike. . .

# ancient scratch

In [ ]:
z_0 = 0.3
def smooth_func(z):
    return 1/(2 * z_0) * (z/z_0)**2 * np.exp(-z/z_0)
zs = np.linspace(0., 1., 100)

nz = smooth_func(zs[:-1])
nz /= np.dot(nz, zs[1:]-zs[:-1])
plt.plot(zs[:-1], nz)

In [ ]:
print(np.dot(true_funcs[0].evaluate(grid_mids), true_nz_info['grid'][1:]-true_nz_info['grid'][:-1]))

In [ ]:
print(np.dot(true_funcs[0].distweights, true_funcs[0].dbins))

In [ ]:
func = sps.norm(0.25, 0.05)
print(func.std())
x = np.linspace(0., 1., 100)
plt.plot(x, func.pdf(x))

In [ ]:
n_bins = 100
z_mids = np.linspace(0., 1., n_bins)

a = 1.
b = 20.#mid-scale variations, larger means more peaks
c = 1.e-6#longest-scale variation, lower increases amplitude relative to small-scale
prior_var = np.eye(n_bins)
for k in range(n_bins):
    prior_var[k] = a * np.exp(-0.5 * b * (z_mids[k] - z_mids) ** 2)
prior_var += c * np.identity(n_bins)

prior_mean = np.zeros(n_bins)
prior = chippr.mvn(prior_mean, prior_var)

In [ ]:
samples = prior.sample(7)
for each in samples:
    plt.plot(z_mids, each)